# 交差検証法

In [1]:
from sklearn.datasets import load_iris
import numpy as np
iris = load_iris()
X = iris.data.astype(np.float32)
y = iris.target

In [3]:
from sklearn.model_selection import train_test_split
X_fold1, X_fold2, y_fold1, y_fold2 = train_test_split(X, y, random_state=37, train_size=0.5)

In [5]:
import cv2
knn = cv2.ml.KNearest_create()
knn.setDefaultK(1)

In [6]:
knn.train(X_fold1, cv2.ml.ROW_SAMPLE, y_fold1)
_, y_hat_fold2 = knn.predict(X_fold2)
knn.train(X_fold2, cv2.ml.ROW_SAMPLE, y_fold2)
_, y_hat_fold1 = knn.predict(X_fold1)

In [7]:
from sklearn.metrics import accuracy_score
accuracy_score(y_fold1, y_hat_fold1)

0.92

In [8]:
accuracy_score(y_fold2, y_hat_fold2)

0.88

#### k分割交差検証法にscikit-learnを利用する

In [9]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)

In [10]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=5)
scores

array([0.96666667, 0.96666667, 0.93333333, 0.93333333, 1.        ])

In [12]:
scores.mean(), scores.std()

(0.96, 0.024944382578492935)

#### Leave-One-Out交差検証法

In [13]:
from sklearn.model_selection import LeaveOneOut

In [15]:
scores = cross_val_score(model, X, y, cv=LeaveOneOut())

In [16]:
scores

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [19]:
scores.mean(), scores.std()

(0.96, 0.19595917942265423)

In [20]:
import cv2
knn = cv2.ml.KNearest_create()
knn.setDefaultK(1)

In [21]:
import numpy as np
idx_boot = np.random.choice(len(X), size=len(X), replace=True)
X_boot = X[idx_boot, :]
y_boot = y[idx_boot]

In [27]:
idx_oob = np.array([x not in idx_boot for x in np.arange(len(X))], dtype=bool)
X_oob = X[idx_oob, :]
y_oob = y[idx_oob]

In [29]:
knn.train(X_boot, cv2.ml.ROW_SAMPLE, y_boot)

True

In [30]:
_, y_hat = knn.predict(X_oob)
accuracy_score(y_oob, y_hat)

0.9444444444444444

In [33]:
def yield_bootstrap(model, X, y, n_iter=10000):
    for _ in range(n_iter):
        idx_boot = np.random.choice(len(X), size=len(X), replace=True)
        X_boot = X[idx_boot, :]
        y_boot = y[idx_boot]
        knn.train(X_boot, cv2.ml.ROW_SAMPLE, y_boot)
        idx_oob = np.array([x not in idx_boot for x in np.arange(len(X))], dtype=bool)
        X_oob = X[idx_oob, :]
        y_oob = y[idx_oob]
        _, y_hat = knn.predict(X_oob)
        yield accuracy_score(y_oob, y_hat)

In [34]:
np.random.seed(42)

In [35]:
list(yield_bootstrap(knn, X, y, n_iter=10))

[0.9833333333333333,
 0.9365079365079365,
 0.9245283018867925,
 0.9230769230769231,
 0.9454545454545454,
 0.9473684210526315,
 0.9814814814814815,
 0.9607843137254902,
 0.9322033898305084,
 0.9661016949152542]

In [37]:
acc = list(yield_bootstrap(knn, X, y, n_iter=1000))
np.mean(acc), np.std(acc)

(0.955241551364192, 0.022040380995646654)

In [42]:
acc = list(yield_bootstrap(knn, X, y, n_iter=10000))
np.mean(acc), np.std(acc)

(0.9547852370336725, 0.0215960289173746)

# 誤差結果の有意性の検証

In [43]:
from scipy.stats import ttest_ind
scores_a = [1, 1, 1, 1, 1]
scores_b = [0, 0, 0, 0, 0]

In [44]:
ttest_ind(scores_a, scores_b)

Ttest_indResult(statistic=inf, pvalue=0.0)

In [45]:
scores_a = [0.9, 0.9, 0.9, 0.8, 0.8]
scores_b = [0.8, 0.8, 0.9, 0.9, 0.9]
ttest_ind(scores_a, scores_b)

Ttest_indResult(statistic=0.0, pvalue=1.0)

In [46]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
k1 = KNeighborsClassifier(n_neighbors=1)
scores_k1 = cross_val_score(k1, X, y, cv=10)
np.mean(scores_k1), np.std(scores_k1)

(0.96, 0.05333333333333332)

In [48]:
k3 = KNeighborsClassifier(n_neighbors=3)
scores_k3 = cross_val_score(k3, X, y, cv=10)
np.mean(scores_k3), np.std(scores_k3)

(0.9666666666666666, 0.04472135954999579)

In [50]:
ttest_ind(scores_k1, scores_k3)

Ttest_indResult(statistic=-0.2873478855663425, pvalue=0.7771278487505296)

#### マクネマー検定

In [51]:
scores_a = np.array([1, 1, 1, 1, 1])
scores_b = np.array([0, 0, 0, 0, 0])

In [52]:
a1_b0 = scores_a * (1 - scores_b)
a1_b0

array([1, 1, 1, 1, 1])

In [54]:
a0_b1 = (1 - scores_a) * scores_b
a0_b1

array([0, 0, 0, 0, 0])

In [61]:
from scipy.stats import binom
def mcnemar_midp(b, c):
    n = b + c
    x = min(b, c)
    dist = binom(n, .5)
    p = 2. * dist.cdf(x)
    midp = p - dist.pmf(x)
    return midp

In [62]:
mcnemar_midp(a1_b0.sum(), a0_b1.sum())

0.03125

In [63]:
scores_k1 = cross_val_score(k1, X, y, cv=LeaveOneOut())
scores_k3 = cross_val_score(k3, X, y, cv=LeaveOneOut())

In [64]:
np.sum(scores_k1 * (1 - scores_k3))

0.0

In [65]:
np.sum((1 - scores_k1) * scores_k3)

0.0

In [67]:
mcnemar_midp(np.sum(scores_k1 * (1 - scores_k3)), np.sum((1 - scores_k1) * scores_k3)

SyntaxError: unexpected EOF while parsing (4201556734.py, line 1)